In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("srinivasanusuri/optimization-of-machine-downtime")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/srinivasanusuri/optimization-of-machine-downtime/versions/1


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.metrics
import numpy as np
import datetime

In [ ]:
df = pd.read_csv(f"{path}/Machine Downtime.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.head()

<ipython-input-77-065348adfd26>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


,Date,Machine_ID,Assembly_Line_No,Hydraulic_Pressure(bar),Coolant_Pressure(bar),Air_System_Pressure(bar),Coolant_Temperature,Hydraulic_Oil_Temperature(?C),Spindle_Bearing_Temperature(?C),Spindle_Vibration(?m),Tool_Vibration(?m),Spindle_Speed(RPM),Voltage(volts),Torque(Nm),Cutting(kN),Downtime
0,2021-12-31,Makino-L1-Unit1-2013,Shopfloor-L1,71.04,6.933725,6.284965,25.6,46.0,33.4,1.291,26.492,25892.0,335.0,24.055326,3.58,Machine_Failure
1,2021-12-31,Makino-L1-Unit1-2013,Shopfloor-L1,125.33,4.936892,6.196733,35.3,47.4,34.6,1.382,25.274,19856.0,368.0,14.202890,2.68,Machine_Failure
2,2021-12-31,Makino-L3-Unit1-2015,Shopfloor-L3,71.12,6.839413,6.655448,13.1,40.7,33.0,1.319,30.608,19851.0,325.0,24.049267,3.55,Machine_Failure
3,2022-05-31,Makino-L2-Unit1-2015,Shopfloor-L2,139.34,4.574382,6.560394,24.4,44.2,40.6,0.618,30.791,18461.0,360.0,25.860029,3.55,Machine_Failure
4,2022-03-31,Makino-L1-Unit1-2013,Shopfloor-L1,60.51,6.893182,6.141238,4.1,47.3,31.4,0.983,25.516,26526.0,354.0,25.515874,3.55,Machine_Failure


In [ ]:
df['Machine_ID'].value_counts()

,count
Machine_ID,
Makino-L1-Unit1-2013,874
Makino-L3-Unit1-2015,818
Makino-L2-Unit1-2015,808


In [ ]:
df['Assembly_Line_No'].value_counts()

,count
Assembly_Line_No,
Shopfloor-L1,874
Shopfloor-L3,818
Shopfloor-L2,808


In [ ]:
pd.crosstab(df['Machine_ID'], df['Assembly_Line_No'])

Assembly_Line_No,Shopfloor-L1,Shopfloor-L2,Shopfloor-L3
Machine_ID,,,
Makino-L1-Unit1-2013,874,0,0
Makino-L2-Unit1-2015,0,808,0
Makino-L3-Unit1-2015,0,0,818


> This means that we can discard one column, as the machine ID comes from the Assembly Line

In [ ]:
df = df.drop(['Assembly_Line_No'], axis=1)

In [ ]:
one_hot_encoder = sklearn.preprocessing.OneHotEncoder(sparse_output=False)
one_hot_encoder.set_output(transform='pandas')

df = pd.concat(
  [
    df.drop(['Machine_ID'], axis=1),
    one_hot_encoder.fit_transform(df[['Machine_ID']])
  ],
  axis=1)
df.tail()

,Date,Hydraulic_Pressure(bar),Coolant_Pressure(bar),Air_System_Pressure(bar),Coolant_Temperature,Hydraulic_Oil_Temperature(?C),Spindle_Bearing_Temperature(?C),Spindle_Vibration(?m),Tool_Vibration(?m),Spindle_Speed(RPM),Voltage(volts),Torque(Nm),Cutting(kN),Downtime,Machine_ID_Makino-L1-Unit1-2013,Machine_ID_Makino-L2-Unit1-2015,Machine_ID_Makino-L3-Unit1-2015
2495,2022-02-01,112.715506,5.220885,6.196610,22.3,48.8,37.2,0.910,20.282,20974.0,282.0,22.761610,2.72,No_Machine_Failure,1.0,0.0,0.0
2496,2022-02-01,103.086653,5.211886,7.074653,11.9,48.3,31.5,1.106,34.708,20951.0,319.0,22.786597,2.94,No_Machine_Failure,1.0,0.0,0.0
2497,2022-02-01,118.643165,5.212991,6.530049,4.5,49.9,36.2,0.288,16.828,20958.0,335.0,22.778987,NaN,No_Machine_Failure,0.0,1.0,0.0
2498,2022-02-01,145.855859,5.207777,6.402655,12.2,44.5,32.1,0.995,26.498,20935.0,376.0,22.804012,2.79,No_Machine_Failure,0.0,0.0,1.0
2499,2022-02-01,96.690000,5.936610,7.109355,29.8,53.2,36.2,0.840,31.580,23576.0,385.0,24.409551,3.55,Machine_Failure,0.0,1.0,0.0


> How do we deal with the date

As maintenance failures likely increase over time, we will encode it as a number of days after the manufacture date

In [ ]:
# df['Manufacture_Date'] = datetime.datetime(2015, 1, 1)
# df.loc[df['Machine_ID_Makino-L1-Unit1-2013'] == 1, 'Manufacture_Date'] = 2013

df['Manufacture_Date'] = np.select([
    df['Machine_ID_Makino-L1-Unit1-2013'] == 1
], [
    pd.to_datetime("1/1/2013")
], default=pd.to_datetime("1/1/2015"))

df['Date'] = df['Date'] - df['Manufacture_Date']

In [ ]:
df

,Date,Hydraulic_Pressure(bar),Coolant_Pressure(bar),Air_System_Pressure(bar),Coolant_Temperature,Hydraulic_Oil_Temperature(?C),Spindle_Bearing_Temperature(?C),Spindle_Vibration(?m),Tool_Vibration(?m),Spindle_Speed(RPM),Voltage(volts),Torque(Nm),Cutting(kN),Downtime,Machine_ID_Makino-L1-Unit1-2013,Machine_ID_Makino-L2-Unit1-2015,Machine_ID_Makino-L3-Unit1-2015,Manufacture_Date
0,3286 days,71.040000,6.933725,6.284965,25.6,46.0,33.4,1.291,26.492,25892.0,335.0,24.055326,3.58,Machine_Failure,1.0,0.0,0.0,2013-01-01
1,3286 days,125.330000,4.936892,6.196733,35.3,47.4,34.6,1.382,25.274,19856.0,368.0,14.202890,2.68,Machine_Failure,1.0,0.0,0.0,2013-01-01
2,2556 days,71.120000,6.839413,6.655448,13.1,40.7,33.0,1.319,30.608,19851.0,325.0,24.049267,3.55,Machine_Failure,0.0,0.0,1.0,2015-01-01
3,2707 days,139.340000,4.574382,6.560394,24.4,44.2,40.6,0.618,30.791,18461.0,360.0,25.860029,3.55,Machine_Failure,0.0,1.0,0.0,2015-01-01
4,3376 days,60.510000,6.893182,6.141238,4.1,47.3,31.4,0.983,25.516,26526.0,354.0,25.515874,3.55,Machine_Failure,1.0,0.0,0.0,2013-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,3318 days,112.715506,5.220885,6.196610,22.3,48.8,37.2,0.910,20.282,20974.0,282.0,22.761610,2.72,No_Machine_Failure,1.0,0.0,0.0,2013-01-01
2496,3318 days,103.086653,5.211886,7.074653,11.9,48.3,31.5,1.106,34.708,20951.0,319.0,22.786597,2.94,No_Machine_Failure,1.0,0.0,0.0,2013-01-01
2497,2588 days,118.643165,5.212991,6.530049,4.5,49.9,36.2,0.288,16.828,20958.0,335.0,22.778987,NaN,No_Machine_Failure,0.0,1.0,0.0,2015-01-01
2498,2588 days,145.855859,5.207777,6.402655,12.2,44.5,32.1,0.995,26.498,20935.0,376.0,22.804012,2.79,No_Machine_Failure,0.0,0.0,1.0,2015-01-01


# Actual Training

In [ ]:
X, y = df.drop(['Downtime'], axis=1), df['Downtime']

In [ ]:
X['Date'] = X['Date'].dt.days

In [ ]:
X = X.drop('Manufacture_Date', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# clf = sklearn.ensemble.RandomForestClassifier(max_depth=5, random_state=0)

clf = sklearn.pipeline.make_pipeline(
    sklearn.preprocessing.StandardScaler(),
    sklearn.ensemble.RandomForestClassifier(max_depth=6, random_state=0)
)

clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, random_state=0))])

In [ ]:
clf.score(X_test, y_test)

0.9792

In [ ]:
sklearn.metrics.confusion_matrix(y_test, clf.predict(X_test))

array([[309,   7],
       [  6, 303]])